In [1]:
import numpy as np
import copy

### MDP setting
* state: index of each arrays
  * 0: Home, 1: Office, 2: Bar
* MDP array describes the probabilities for move to next state if choiced moving

In [2]:
p = [0.8, 0.5, 1.0]

### Discount rate

In [3]:
gamma = 0.95

### Reward $r(s'| s, a)$
* 1st index: $s'$
* 2nd index: $s$
* 3rd index: $a = True(1) or False(0)$

In [4]:
r = np.zeros((3, 3, 2))
r[0, 1, 0] = 1.0
r[0, 2, 0] = 2.0
r[0, 0, 1] = 0.0
r[1, 0, 0] = 1.0
r[1, 2, 0] = 2.0
r[1, 1, 1] = 1.0
r[2, 0, 0] = 1.0
r[2, 1, 0] = 0.0
r[2, 2, 1] = -1.0

### Initial Value Function $v_{\pi_0}$

### Initial action Value funcion $q_{\pi_0}$

In [17]:
q = np.zeros((3, 2))

### initail policy distribution $\pi_0$
* the array describes the probabilities for choice of move

In [18]:
pi = [0.5, 0.5, 0.5]

### definition of policy estimation

In [8]:
def policy_estimator(pi, p, r, gamma):
    # initialize
    # R: reward expection conditioned by initial state
    # P: transtion probability between the states
    
    R = [0, 0, 0]
    P = np.zeros((3, 3))

    for i in range(3):

        # 状態遷移行列の計算
        P[i, i] = 1 - pi[i]
        P[i, (i + 1) % 3] = p[i] * pi[i]
        P[i, (i + 2) % 3] = (1 - p[i]) * pi[i]

        # 報酬ベクトルの計算
        R[i] = pi[i] * (p[i] * r[i, (i + 1) % 3, 0] +
                        (1 - p[i]) * r[i, (i + 2) % 3, 0]
                        ) + (1 - pi[i]) * r[i, i, 1]

    # 行列計算によるベルマン方程式の求解
    A = np.eye(3) - gamma * P
    B = np.linalg.inv(A)
    v_sol = np.dot(B, R)

    return v_sol

### iteration

In [20]:
v = [0, 0, 0]
v_pev = copy.copy(v)

for step in range(100):

    # 方策評価ステップ
    v = policy_estimator(pi, p, r, gamma)

    # 価値関数 v が前ステップの値 v_prep を改善しなければ終了
    if np.min(v - v_prev) <= 0:
        break

    # 現ステップの価値関数と方策を表示
    print('step:', step, ' value:', v, ' policy:', pi)

    # 方策改善ステップ
    for i in range(3):

        # 行動価値関数を計算
        q[i, 0] = p[i] * (
            r[i, (i + 1) % 3, 0] + gamma * v[(i + 1) % 3]
        ) + (1 - p[i]) * (r[i, (i + 2) % 3, 0]
                          + gamma * v[(i + 2) % 3])
        q[i, 1] = r[i, i, 1] + gamma * v[i]

        # 行動価値関数のもとで greedy に方策を改善
        if q[i, 0] > q[i, 1]:
            pi[i] = 1
        elif q[i, 0] == q[i, 1]:
            pi[i] = 0.5
        else:
            pi[i] = 0

    # 現ステップの価値関数を記録
    v_prev = copy.copy(v)
